# Things left to do on model front

    1) Either use original polyglot embeddings or original code on word2vec to make sure our model is getting similar results; would be really dumb if we have a small mistake in our model and aren't actually training it correctly

    2) Build 1-2 more models of increasing complexity.  Perhaps we add attention to one of the biLSTM layers or attempt encoder decoder approach.
    
    3) Build script that takes a feature and works through getting avg. embeddings

In [1]:
# Useful library for embeddings

import class_lstm
from class_lstm import MimicLSTM

import word2vec_preprocess
import word2vec_preprocess as preprocess

import gensim
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import importlib

from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing import sequence 
import tensorflow as tf

import gensim

word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    './word2vec_model/GoogleNews-vectors-negative300.bin', binary=True) 


/home/cgleach/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def nn_word(word, word_emb_dict, lstm_model, char_dict, word2vec):
    indices = np.array([char_dict[letter] for letter in word])
    indices = sequence.pad_sequences([indices], maxlen=25) 
    prediction = lstm_model.predict(tf.reshape(indices,(1,-1)), steps=1)
    
    actual = np.reshape(word2vec[word],(1,-1))
    actual_sim = cosine_similarity(actual,prediction)
    print("Actual sim is %s" %actual_sim)
    
    best_word = ''
    best_sim = 0
    total_sim = 0
    total_count = 0
    
    for k,v in word_emb_dict.items():
        
        v = np.reshape(v,(1,-1))
        sim = cosine_similarity(prediction, v)
        total_sim += sim
        total_count += 1
        
        if sim > best_sim:
            best_word = k
            best_sim = sim
            print(best_word, best_sim)
    avg_sim = total_sim/total_count
    
    print("actual sim was %s while avg was %s" %(actual_sim,avg_sim))
    return actual_sim, avg_sim

In [3]:
data_dict, chardict = preprocess.run()

In [4]:
layers = 1
H = 50
character_dim = 20
output_dim = 300

In [5]:
# Reload our packages as we test 
importlib.reload(class_lstm)
importlib.reload(word2vec_preprocess)

<module 'word2vec_preprocess' from '/home/cgleach/w266/final_project/w266-final-project-OOV-fake-bias-detection/original_work/word2vec_preprocess.py'>

In [6]:
model = MimicLSTM(layers, H, chardict, character_dim, data_dict, 
                  batch_size=1000, epochs=10, full_set=True)

model.save_model("./mimic_model_original.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 20)            1120      
_________________________________________________________________
bidirectional (Bidirectional (None, 100)               28400     
_________________________________________________________________
dense (Dense)                (None, 300)               30300     
Total params: 59,820
Trainable params: 59,820
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
760310/760310 [==============================] - 119s 157us/sample - loss: 0.0193
Epoch 2/10
760310/760310 [==============================] - 117s 154us/sample - loss: 0.0190
Epoch 3/10
760310/760310 [======================

'Save Succesful'

In [9]:
a,b = nn_word("compartmentalize", data_dict, model.lstm, chardict, word2vec)


Actual sim is [[0.29064178]]
in [[0.08038858]]
that [[0.2208077]]
or [[0.23610187]]
government [[0.24608949]]
such [[0.2760615]]
these [[0.30484337]]
products [[0.34159136]]
technology [[0.34362707]]
product [[0.35605633]]
technologies [[0.38568747]]
processes [[0.39017558]]
proprietary [[0.42891064]]
functional [[0.4320678]]
efficacy [[0.43297523]]
subjective [[0.48869294]]
synthesis [[0.49266905]]
vivo [[0.50209945]]
Datafeed [[0.5150604]]
ProQuote [[0.53543985]]
olp [[0.5482096]]
defaultHeight [[0.5899529]]
pusporte [[0.60609496]]
Nathesh [[0.6230533]]
glycosylated [[0.6605201]]
PittsburgMo [[0.6821924]]
PittsburgKan [[0.70375466]]
MookieTheCat [[0.7224848]]
actual sim was [[0.29064178]] while avg was [[0.19018801]]


In [10]:
layers = 2
H = 100
character_dim = 50
output_dim = 300

In [11]:
model_complex = MimicLSTM(layers, H, chardict, character_dim, data_dict, batch_size=1000, 
                          epochs = 25, full_set = True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 50)            2800      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 200)           120800    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 300)               60300     
Total params: 424,700
Trainable params: 424,700
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
760310/760310 [==============================] - 741s 975us/sample - loss: 0.0192
Epoch 2/25
760310/760310 [==============================] - 728s 958us/sample - loss: 0.0189
Epoch 3/25
760310/760310 [==============================] - 728s 957us/sample - lo

In [12]:
model.save_model("./mimic_model_complex.h5")

'Save Succesful'

In [22]:
a,b = nn_word("crying", data_dict, model_complex.lstm, chardict, word2vec)


Actual sim is [[0.44117203]]
in [[0.15407538]]
for [[0.15527652]]
that [[0.1856101]]
on [[0.18662052]]
the [[0.23123805]]
not [[0.24033497]]
their [[0.27377588]]
you [[0.30320066]]
all [[0.30395806]]
do [[0.34623665]]
just [[0.4128584]]
getting [[0.47810704]]
talking [[0.54108655]]
putting [[0.54895616]]
ripping [[0.6050508]]
messing [[0.6262998]]
actual sim was [[0.44117203]] while avg was [[0.11221445]]


In [ ]:
# Add attention layer after first bidirectional layers + dropout after final
model_complex_attention = MimicLSTM(layers, H, chardict, character_dim, data_dict, batch_size=1000, 
                          epochs = 25, full_set = True, attention=True)